## MANP problem (*M*aximizing *A*verage *N*umber of *P*ages read in multi-page publications) as an MDP

In this section we will present MANP problem as MDP. In order to simplify the representation at this stage we add two constraints on the initial model:
1. On each step the agent has only two possible actions - "next" and "leave".
2. The agent can't visit the same page twice.

The problem remains the same: given a set of content units (pages)  $B = \{c_1\dots c_K\}$, we want to present it to users in such a way that will maximize the average number of pages viewed. The only thing we can change is an order of the representation for each particular user.<br>

Even though the naive and straightforward way to define the MDP is to set the content consumer being an agent, we will do it in a different way. The content producer will be an agent and the Web resource visitor will be the part of the environment. Such a formulation allows us to use classic tools of planning to optimize the target MANP problem's value.




<hr style="border:2px solid gray"> </hr>

**Here is a definition of the MDP:**<br>

 1. State space $S = S' \cup \{s_{init},s_{final}\}$ : 
    * There will be an initial state $s_{init}$. The visitor haven't seen yet any content.
    * Each state $s_j = C_k \in S'$ will contain all the pages the user saw $C_k \subset B$ (nodes the user visited).
    * There will be a final state $s_{final}$. It represents the fact that the user finished watching the pages.
 <br> <br> 
 2.  Action space $A=\{a_{c_1}\dots a_{c_k}\}$. Action $a_{c_k}$ denotes the action of presenting  to the visitor page $c_k \in B$.
 <br><br>
 3.  Reward function $R(a,s,s')$ for $a \in A\;s,s'\in S$  depends only on the resulting state $s'$ :
     * If the agent traversed to $s_{final}$, the reward is zero. Otherwise it's 1.<br>
        $\forall a\in A\;s_i\in S :R(a,s_i,s_{final}) = 0$, otherwise $R(a,s,s')=1$
 <br><br>
 4. Transition function $T(a,s,s')$ depends only on action $a$ and it is not known to us. We will denote it as $T'(a) = T(a,s,s')$<br>
   It actually determines possible reactions of the content consumer to the page $c_i$ (or for action $a_{c_i}$ in terms of MDP) , the probability that the user press "next" or "leave". <br>
   At the next stages we may assume that it depends also on $s$ ,i.e. visitors behavior depends on the pages seen.
<hr style="border:2px solid gray"> </hr>

## Implementing the MDP

<span style='background:#D3D3D3'> POMDPs.jl </span> package will be used to help modeling the MDP. It provides flexible interface that is compatable with different tools for solving and simulating an MDP. It specifies the set of functions that should be implemented. In fact we overload the methods of POMDPs package to work with the process we define. <<< link to the page of mdp>>>

In following several subsections we implement this methods one by one and provide a short explanation for the  implementation.

In [19]:

try
    using Pkg
    using POMDPs            # POMDPs.jl interface
    using POMDPModelTools   # POMDPModelTools has tools that help build the MDP definition
    using POMDPPolicies
    using POMDPSimulators
    using Combinatorics
    using DiscreteValueIteration
catch  LoadError
    Pkg.add("POMDPs")
    Pkg.add("POMDPModelTools")
    Pkg.add("POMDPPolicies")
    Pkg.add("POMDPSimulators")
    Pkg.add("Combinatorics")
    Pkg.add("DiscreteValueIteration")
    using DiscreteValueIteration
    using POMDPs
    using POMDPModelTools
    using Combinatorics
end;

#### Basic objects
First two basic objects for the model are **Page** and **User1**. 

 <span style='background:#D3D3D3'> User1 </span> object will encapsulate the particular user's behavior.
Although the user is a part of an environment in this model, and it may be fully described by the transition function of the process , we decided to represent it as a separate object. It was done for the several reasons: 1) to preserve the initial structure of MANP problem , where the user is an essential part  2) to have a convenient ability to change the user's behavior i.e simulating the arrival of different types of users 3) reusing the same object in other models for MANP problem.

Page is the object that represents the unit of content which the user consumes (watches) before deciding on her next action (leave or proceed) 



In [20]:
struct Page
   id :: Int64 
end

pages_collection = nothing

struct User1 
   id :: Int64
   userPreferencesFunction:: Any   # Recieve pages visited, current page shown
end;


#### Auxiliary functions

In [21]:
#
function CreatePages(number :: Int64)
    global pages_collection = [Page(id) for id in 1:number]
end

function bitarr_to_int(arr)
    return sum(arr .* (2 .^ collect(0:length(arr)-1)))
    end;



### MDP basic elements



In this section we define the parts of the MDP: actions , states and the mdp object itself.
We remind that our representation, when the user is part of the environment and the content provider is an agent, may seem counterintuitive at first sight. But it makes sense supposing that the goal is to maximize the number of pages viewed, which is a content provider's goal.

The <span style='background:#D3D3D3'> Action1</span>  object represents the action of presenting to the user a particular unit of content (aka page) .  
The <span style='background:#D3D3D3'> State1</span> object specifies state that contains the pages that the user already have seen and the indicator whether it is a terminal state.

In [22]:
struct Action1
   show_page :: Union{Page,Nothing}           # The page that is presented to the user
end

struct State1
    visited_pages :: Any                      # Didn't specified, because we need here immutable type.                                              # Supposed to be an immutable set
    if_terminal :: Bool
end 

#### MDP object

As a part of the POMDPs interface specifications , we should define an MDP object that inherits from the abstract type MDP and specify the type of the actions' and states' object. The structure <span style='background:#D3D3D3'> ContentProducerMdp </span> contains the parameters of the MDP environment. It is passed as a first argument to all the POMDPs interface's functions to navigate to the implementations that was written for this concrete MDP. 

It will contain a current user that determine the process dynamics, array of content units (pages) as well as tools for tracking and statistics.   

In [23]:
struct MdpStatistics1
   current_user_path :: Array{Page,1} 
   nexts_per_page :: Array{Int64,1}
    leaves_per_page :: Array{Int64,1}
    total_pages_seen :: Int64
    users_number :: Int64
    
end

mutable struct ContentProducerMdp <: MDP{State1,Action1}
    pages :: Array{Page,1}
    current_user :: User1
    statistics :: MdpStatistics1
    
end



##### Auxiliary functions
Return the array of all the state and actions

In [24]:
function POMDPs.states(mdp::ContentProducerMdp)
    pages_superset = collect(powerset(mdp.pages))             # We assume that the function preserves the order (not in docs)
    states = [State1(Tuple(pages),false) for pages in pages_superset]  
    push!(states, State1(Tuple([]),true))
    return states
end

POMDPs.isterminal(mdp::ContentProducerMdp,s::State1)= return s.if_terminal

function POMDPs.actions(mdp::ContentProducerMdp)
    actions = [Action1(page) for page in mdp.pages]  
end

#### Transition function

Transition function returns distribution on states. In current less complicated formulation of the problem, the transition distribution depends only on the action. The user decision whether to proceed or to leave explained only by page that was presented him last.

Exactly two states have non-zero probabilities given action $a_i$. They are 1) the final state and 2) the state that contains all the pages the user visited including the last one $c_i$. The distribution on these states is defined by user's leaving probability for each page. The transition function queries the current user (userPreferencesFunction attribute of User1) , that appears in the MDP object for it. <br>
Special case is when the visitor already saw all the pages, and it is automatically traversed to the final state. 


In [25]:
function POMDPs.transition(mdp::ContentProducerMdp,state::State1,act::Action1)

    pages_visited = state.visited_pages
    current_page = act.show_page
    
    if current_page in pages_visited
        return SparseCat([State1(Tuple([]),true)], [1.0]) 
    end
    
    if POMDPs.isterminal(mdp,state)
       return SparseCat([State1(Tuple([]),true)], [1.0]) 
    end
    
    if length(pages_visited) == length(mdp.pages) - 1
       return SparseCat([State1(Tuple([]),true)], [1.0])
    end

    leaving_probab = mdp.current_user.userPreferencesFunction(current_page,pages_visited)
    
       if length(pages_visited) == 0
        new_page_visited = [current_page]
    else

        new_page_visited = push!(collect(pages_visited),current_page)
    end
    sort!(new_page_visited,by=page->page.id,alg=InsertionSort)  # The array is almost sorted , that's why insertion sort
    
    return SparseCat([State1(Tuple(new_page_visited),false), State1(Tuple([]),true)],[1-leaving_probab,leaving_probab])
end

#### Reward function

The reward function is also simple and mostly depends on the destination state $s'$. If $s'$ is a final state, it means that the user leaved after seeing the last content unit and the reward is zero, and if $s'$ is non-terminal state then the user pressed "next" and the reward is 1.
The only case when the reward is positive when the visitor arrived at the final state is when she saw all the pages.
Remark:
In addition -100 reward was added when the page that was already seen in proposed to the user again, it was done for out of the box POMDPs solvers, to prevent revisiting the pages.


The reward is in fact a number of pages that the visitor saw before leaving, that is exactly the MANP objective.

In [26]:
function POMDPs.reward(mdp::ContentProducerMdp,state::State1,act::Action1,stateP::State1)
    is_all_pages_seen = (POMDPs.isterminal(mdp,stateP) 
                        && length(state.visited_pages) >= (length(mdp.pages) - 1))
    
    if act.show_page in state.visited_pages
        return -100
    elseif !POMDPs.isterminal(mdp,stateP) ||  is_all_pages_seen
        return 1
    else 
        return 0
    end

end

##### Auxiliary functions

Defining the initial state and the discount factor. 

In [27]:
POMDPs.discount(mdp::ContentProducerMdp) = 1

POMDPs.initialstate(mdp::ContentProducerMdp)= SparseCat([State1(Tuple([]),false)],[1])


The POMDPs interface requires also implementing indexing function for states and actions. Here we assign a number for each state and action. State's index is actually a binary encoding of what pages has been visited. And for actions it's the page id.

In [28]:
function POMDPs.stateindex(mdp::ContentProducerMdp, state::State1)
    indeces = zeros(Int64,length(mdp.pages))
    i = 1
    visited_pages = state.visited_pages
    for j in 1:length(mdp.pages)
        if i > length(visited_pages)
           break 
        end
        if mdp.pages[j] == visited_pages[i]
           
            indeces[j] = 1
            i+=1
        end
    end
    
    if POMDPs.isterminal(mdp,state)
        return  bitarr_to_int(ones(Int64,length(mdp.pages))) + 2
    end
    return bitarr_to_int(indeces)+ 1
end

function POMDPs.actionindex(mdp::ContentProducerMdp,act::Action1)
   return act.show_page.id
end

# The following sections are not yet completed 
### Tests of policy run and mdp solving +  mdp  statistics

#### Statistics (*Under construction*)

In [29]:
#
function InitStatistics1(pages_number :: Int64)
    nexts_per_page = zeros(pages_number)
    leaves_per_page = zeros(pages_number)
    
    return MdpStatistics1([],nexts_per_page,leaves_per_page,0,0)
end

function RecordStatistics(mdp::ContentProducerMdp, prev_state::State1,next_state::State1,act::Action1)
    statistics = mdp.statistics
    current_page = act.show_page
    
    is_all_pages_seen = (POMDPs.isterminal(mdp,stateP) 
                        && length(state.visited_pages) >= (length(mdp.pages) - 1))
    
    if act.show_page in state.visited_pages
        push!(statistics.current_user_path, current_page)
        push!(statistics.nexts_per_us)
    elseif !POMDPs.isterminal(mdp,stateP) ||  is_all_pages_seen
        push!(statistics.current_user_path, current_page)
    else 
        push!(statistics.current_user_path, current_page)
    end

end

RecordStatistics (generic function with 1 method)

In [30]:
statistics = InitStatistics1(5)

mdp = ContentProducerMdp(CreatePages(5),User1(1,(x,y)->0.1),statistics)

##
    state = State1(Tuple([]),false)

    action1 = Action1(Page(4))
    # POMDPs.states(mdp)
    # POMDPs.actions(mdp)

    POMDPs.transition(mdp,state,action1)

    POMDPs.reward(mdp,state,action1,state)#State1(Tuple([]),true))
    # POMDPs.stateindex(mdp,state)

    # POMDPs.actionindex(mdp,action1)

    # p = Page(1)

    # State1((Page(2),),true) == State1((Page(1),),true)

    # collect(powerset([:a,1,"dsa"]))

1

#### Random policy + step by step run test (*Under construction*)

In [31]:



policy = RandomPolicy(mdp)

for (s,a,r) in stepthrough(mdp, policy, "s,a,r", max_steps=10)
    @show s
    @show a
    @show r
    println()
end

s = State1((), false)
a = Action1(Page(1))
r = 1

s = State1((Page(1),), false)
a = Action1(Page(1))
r = -100



#### Value iteration solver test (*Under construction*)

In [32]:

POMDPs.isterminal(s::State1)=POMDPs.isterminal(mdp,s)
# initialize the problem

# initialize the solver
# max_iterations: maximum number of iterations value iteration runs for (default is 100)
# belres: the value of Bellman residual used in the solver (defualt is 1e-3)
solver = ValueIterationSolver(max_iterations=10, belres=1e-3; verbose=true)

# solve for an optimal policy
policy = solve(solver, mdp);

[Iteration 1   ] residual:        100 | iteration runtime:      0.719 ms, (  0.000719 s total)
[Iteration 2   ] residual:        0.9 | iteration runtime:      2.106 ms, (   0.00282 s total)
[Iteration 3   ] residual:       0.81 | iteration runtime:      0.644 ms, (   0.00347 s total)
[Iteration 4   ] residual:      0.729 | iteration runtime:      0.658 ms, (   0.00413 s total)
[Iteration 5   ] residual:      0.656 | iteration runtime:      3.286 ms, (   0.00741 s total)
[Iteration 6   ] residual:          0 | iteration runtime:      0.878 ms, (   0.00829 s total)


In [33]:
a = action(policy, State1((Page(1),), false))

Action1(Page(2))

### Transition probabilities are known

Let's imagine the situation when the preferences of the visitor are known, and they are encapsulated in the probability of leaving for every particular page. It completely defines the transition function and the MDP . In this case the  problem is reduced to the planning problem.<br> 
These four characteristics made the current planning problem's solution trivial.
1. The transition function is independent of the pages the user has visited, and depend only on the current page shown. 
2. The states can't be repeated for each particular user, the user can see each page only once.
3. The user can traverse to exactly two states given some action,and one of them is $s_{final}$.
4. The reward depends only on the forthcoming state, and it is equal for all states except for the final.

Let's consider the state value function given any policy $\pi$.<br>
&emsp;$V(s_i) = \sum_{j}T(\pi(s_i),s_i,s_j)( R(\pi(s_i),s_i,s_j) + V(s_j) )$<br>
There is only one possible descending state with non-zero value for each action (The provider presents content to the visitor and she proceeds to the next page) and the reward is 1. As a result we can simplify the value function for our MDP:<br>
&emsp;$V(s_i) = T(\pi(s_i),s_i,s_j)( 1 + V(s_j) )$ ,<br> If we substitute the previous expression for each state into the value function of the initial state, we receive <br> 
&emsp;$V(s_{init}) =  T(\pi(s_{init}),s_{init},s_i)( 1 +  T(\pi(s_i),s_i,s_j)( 1 + \dots ) )$.<br>

Given the fact that: (1) each state can appear only once, (2) the transition probability depends only on the action, we can say that in order to maximize the value of $s_{init}$ (and as a result find an optimal policy) we need to arrange an actions in descending order by there transition probabilities. That is the optimal policy's action in the initial state will be $\arg\max_{\pi(s_{init})\in A} T(\pi(s_{init}),s_{init},s_i)$ and so on.<br>
<br>
As we said before, the planning problem turns to be trivial in case we know all the data.<br>
Now let's consider the case when our simplifying assumptions remain valid, but we don't know user's preferences.  


### Transition probabilities are unknown

Unknown content consumer's preferences mean that we don't know the exact probability of "leaving" for each particular page $c_i \in B$. In terms of the MDP it means that the transition function is unknown. But in fact it is only partially true. Given an action $a_{c_i}$ (showing to the user the page $c_i$), we know exactly the states that the process has non-zero probabilities to traverse to. This is the final state and the state that contains all the pages the user visited including the last one $c_i$. So actually we need to find only the probability of leaving after seeing each page and the MDP will be fully known.<br>
This problem turns to be a learning problem.<br>
Again, the problem's structure together with the assumptions makes it relatively simple learning problem. Despite the fact that there are $2^{|B|} + 2$ states, only actions have an impact on the transition probability. Thus we must estimate only $|B|$ values: the probabilities of leaving for each page. <br>

The simple "model learning -> planning" loop seems appropriate solution to deal it. But even though each step appears to be non-complicated, combining them together arise a fundamental exploration/exploitation dilemma.

